# Задание: 3D-реконструкция, перспектива и эпиполярная геометрия

В этом задании вы реализуете ключевые этапы классической 3D-реконструкции:

- Построение виртуальной сцены и проекция 3D-точек на изображение;
- Моделирование радиальной дисторсии;
- Поиск ключевых точек и построение эпиполярной геометрии;
- Построение карты диспаритета и получение 3D-точек.

Заполните участки кода, помеченные как `# TODO`, и выполните соответствующие вычисления.


## Импорты и базовые функции визуализации

Выполните следующую ячейку для подключения библиотек и определения вспомогательных функций.


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (6, 4)
plt.rcParams['figure.dpi'] = 120

def show_img(img, title=None, cmap=None, figsize=(6,4)):
    plt.figure(figsize=figsize)
    if img.ndim == 2 or cmap is not None:
        plt.imshow(img, cmap=cmap)
    else:
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    if title:
        plt.title(title)
    plt.show()


## Построение виртуальной сцены и проекция

Создадим виртуальную камеру и куб в 3D-пространстве. Необходимо реализовать функцию `project_points_cam`, которая проецирует 3D-точки на изображение.

Напомним, что проекция точки \( (X, Y, Z) \) камеры даёт нормированные координаты \( (x, y) = (X/Z, Y/Z) \), а пиксельные координаты находятся по формулам:

$$
u = f_x \cdot x + c_x, \quad v = f_y \cdot y + c_y
$$

Заполните соответствующий участок кода.


In [ ]:
def project_points_cam(K, pts_cam):
    """
    Перспективная проекция 3D-точек (в координатах камеры) на изображение.

    Параметры:
        K: 3x3 — матрица внутренних параметров камеры
        pts_cam: Nx3 — массив 3D-точек в координатах камеры

    Возвращает:
        Nx2 — пиксельные координаты на изображении
    """
    pts_cam = np.asarray(pts_cam, dtype=np.float32)
    X = pts_cam[:, 0]
    Y = pts_cam[:, 1]
    Z = pts_cam[:, 2]

    # TODO: избегайте деления на 0
    Z = np.where(Z == 0, 1e-6, Z)

    # TODO: нормализованные координаты
    x = X / Z
    y = Y / Z

    fx = K[0, 0]
    fy = K[1, 1]
    cx = K[0, 2]
    cy = K[1, 2]

    # TODO: преобразование в пиксели
    u = fx * x + cx
    v = fy * y + cy

    return np.stack([u, v], axis=1)


## Генерация объектов сцены: куб и пол

Здесь мы создаём массивы 3D-точек, представляющих куб и плоскость пола.
Эти точки уже заданы в системе координат камеры и будут использованы
для перспективной проекции на изображение.

Редактировать этот блок не требуется.


In [ ]:
def make_cube_cam(side=2.0, center=(0.0, 0.0, 6.0)):
    cx, cy, cz = center
    s = side / 2
    xs = [-s, s]
    ys = [-s, s]
    zs = [-s, s]
    pts = []
    for x in xs:
        for y in ys:
            for z in zs:
                pts.append([cx + x, cy + y, cz + z])
    return np.array(pts, dtype=np.float32)


def make_floor_cam(size=20.0, n=30, z=10.0):
    xs = np.linspace(-size, size, n)
    zs = np.linspace(0, size * 2, n)
    pts = []
    for x in xs:
        for zz in zs:
            pts.append([x, 0.0, z + zz])
    return np.array(pts, dtype=np.float32)


## Реализация поворота сцены вокруг оси Y

Для наглядной демонстрации перспективных искажений реализуем поворот объекта
(например, куба) вокруг вертикальной оси Y. Это поможет понять, как меняется
внешний вид сцены при изменении ориентации.

Задание: реализуйте матрицу поворота вокруг оси Y.


In [ ]:
def rot_y(angle_deg):
    """
    Возвращает матрицу поворота вокруг оси Y на заданный угол (в градусах).
    """
    # TODO: переведите угол в радианы
    a = np.deg2rad(angle_deg)
    ca = np.cos(a)
    sa = np.sin(a)

    # TODO: соберите и верните матрицу поворота
    return np.array([
        [ca,  0, sa],
        [ 0,  1,  0],
        [-sa, 0, ca]
    ], dtype=np.float32)


## Визуализация сцены: куб в перспективе

Скомбинируем куб, пол и виртуальную камеру в функции `render_scene`.

Ваше задание:
- Проецировать 3D-точки пола и куба на изображение;
- Нарисовать их с помощью OpenCV-функций;
- Повернуть куб перед проекцией (с помощью `rot_y`).

Ниже подготовлен шаблон функции.


In [ ]:
W, H = 512, 384
background = np.full((H, W, 3), 230, dtype=np.uint8)

CUBE_EDGES = [
    (0, 1), (1, 3), (3, 2), (2, 0),
    (4, 5), (5, 7), (7, 6), (6, 4),
    (0, 4), (1, 5), (2, 6), (3, 7)
]

cube_cam_base = make_cube_cam(side=3.0, center=(0.0, 0.0, 8.0))
floor_cam     = make_floor_cam(size=25.0, n=40, z=0.0)

def make_K(f=800.0, cx=None, cy=None):
    if cx is None: cx = W / 2
    if cy is None: cy = H / 2
    return np.array([[f, 0, cx],
                     [0, f, cy],
                     [0, 0, 1]], dtype=np.float32)

def render_scene(f: float, yaw_deg: float, title: str):
    K = make_K(f=f, cx=W/2, cy=H/2)
    img = background.copy()

    # TODO: проецируйте пол и куб
    floor_uv = project_points_cam(K, floor_cam)

    center = cube_cam_base.mean(axis=0)
    R_yaw = rot_y(yaw_deg)
    cube_rel = cube_cam_base - center
    cube_rot = (R_yaw @ cube_rel.T).T + center
    cube_uv = project_points_cam(K, cube_rot)

    # TODO: рисуем точки пола
    for (u, v) in floor_uv.astype(int):
        if 0 <= u < W and 0 <= v < H:
            img[v, u] = (200, 200, 200)

    # TODO: рисуем рёбра и точки куба
    cube_uv_int = cube_uv.astype(int)
    for i, j in CUBE_EDGES:
        u1, v1 = cube_uv_int[i]
        u2, v2 = cube_uv_int[j]
        if (0 <= u1 < W and 0 <= v1 < H and
            0 <= u2 < W and 0 <= v2 < H):
            cv2.line(img, (u1, v1), (u2, v2), (0, 0, 255), 2)

    for (u, v) in cube_uv_int:
        if 0 <= u < W and 0 <= v < H:
            cv2.circle(img, (u, v), 4, (0, 0, 255), -1)

    show_img(img[..., ::-1], title=title)


## Попробуйте разные параметры визуализации

Запустите следующий блок, чтобы увидеть, как меняется вид куба при изменении
фокусного расстояния и угла поворота.


In [ ]:
render_scene(f=800, yaw_deg=0,  title="f=800, yaw=0°")
render_scene(f=400, yaw_deg=0,  title="f=400 (широкий угол), yaw=0°")
render_scene(f=800, yaw_deg=30, title="f=800, yaw=30°")


## Моделирование радиальной дисторсии

Оптические системы с линзами часто вносят искажения в изображение.
Наиболее распространённый тип — **радиальная дисторсия**, при которой
пиксели «раздуваются» или «сжимаются» по мере удаления от центра изображения.

Здесь вы реализуете функцию, которая применяет радиальную дисторсию к координатам 2D-точек.

Формула:
$$
x' = x(1 + k_1 r^2), \quad y' = y(1 + k_1 r^2), \quad r^2 = x^2 + y^2
$$

Где $(x, y)$ — нормализованные координаты (до матрицы камеры),
а $(x', y')$ — искажённые координаты.


In [ ]:
def apply_radial_distortion(xy: np.ndarray, k1: float) -> np.ndarray:
    """
    Применяет радиальную дисторсию к 2D-точкам (в нормализованных координатах).

    Параметры:
        xy: Nx2 — массив [x, y] точек в нормализованных координатах (до матрицы K)
        k1: коэффициент радиальной дисторсии

    Возвращает:
        Nx2 — искажённые координаты
    """
    x = xy[:, 0]
    y = xy[:, 1]
    r2 = x**2 + y**2

    # TODO: примените формулу искажений
    factor = 1 + k1 * r2
    x_dist = x * factor
    y_dist = y * factor

    return np.stack([x_dist, y_dist], axis=1)


## Применение дисторсии к изображению сцены

Теперь вы примените радиальную дисторсию ко всем точкам пола и куба:

1. Спроецируйте 3D-точки с помощью камеры;
2. Преобразуйте пиксельные координаты в нормализованные;
3. Примените `apply_radial_distortion`;
4. Пересчитайте обратно в пиксельные координаты;
5. Нарисуйте результат на изображении.

Сравните сцены с и без дисторсии.


In [ ]:
def undistort_uv_to_xy(uv: np.ndarray, K: np.ndarray) -> np.ndarray:
    """
    Переводит пиксельные координаты (u,v) в нормализованные (x,y) с использованием K.

    Параметры:
        uv: Nx2 — пиксельные координаты
        K: 3x3 — матрица камеры

    Возвращает:
        Nx2 — нормализованные координаты
    """
    fx = K[0, 0]
    fy = K[1, 1]
    cx = K[0, 2]
    cy = K[1, 2]

    # TODO: реализуйте обратное преобразование
    x = (uv[:, 0] - cx) / fx
    y = (uv[:, 1] - cy) / fy

    return np.stack([x, y], axis=1)


def distort_scene(K, k1, yaw_deg=0.0, f=800):
    """
    Строит изображение сцены с радиальной дисторсией.

    - Проецирует пол и куб;
    - Преобразует в нормализованные координаты;
    - Применяет искажение;
    - Возвращает изображение.
    """
    img = background.copy()

    # Проекция
    floor_uv = project_points_cam(K, floor_cam)

    center = cube_cam_base.mean(axis=0)
    R_yaw = rot_y(yaw_deg)
    cube_rel = cube_cam_base - center
    cube_rot = (R_yaw @ cube_rel.T).T + center
    cube_uv = project_points_cam(K, cube_rot)

    # Объединяем
    all_uv = np.vstack([floor_uv, cube_uv])
    xy = undistort_uv_to_xy(all_uv, K)

    # TODO: примените дисторсию
    xy_dist = apply_radial_distortion(xy, k1)

    # Обратно в пиксели
    u_dist = xy_dist[:, 0] * K[0, 0] + K[0, 2]
    v_dist = xy_dist[:, 1] * K[1, 1] + K[1, 2]
    uv_dist = np.stack([u_dist, v_dist], axis=1)

    # Визуализация
    for u, v in uv_dist.astype(int):
        if 0 <= u < W and 0 <= v < H:
            img[v, u] = (50, 100, 200)

    return img


## Визуальное сравнение сцены с и без радиальной дисторсии

Попробуйте несколько значений коэффициента дисторсии `k1`:

- $k_1 = 0$ — без искажений;
- $k_1 > 0$ — \"бочкообразная\" дисторсия;
- $k_1 < 0$ — \"подушкообразная\" дисторсия.


In [ ]:
K = make_K(f=800)

img0 = distort_scene(K, k1=0.0)
img1 = distort_scene(K, k1=+0.12)
img2 = distort_scene(K, k1=-0.18)

show_row([img0, img1, img2],
         ["k1 = 0.0", "k1 = +0.12", "k1 = -0.18"], figsize=(12, 4))


## Загрузка изображений и извлечение ключевых точек

Для эпиполярной геометрии нужны пары изображений одной сцены с небольшим параллаксом.

В этом блоке вы:
1. Загрузите изображения (например, `left.jpg`, `right.jpg`);
2. Найдёте ключевые точки с помощью ORB;
3. Найдёте совпадения между ними.

Все эти шаги можно выполнить с помощью OpenCV.


In [ ]:
imgL_gray = cv2.imread("left.jpg", cv2.IMREAD_GRAYSCALE)
imgR_gray = cv2.imread("right.jpg", cv2.IMREAD_GRAYSCALE)

assert imgL_gray is not None and imgR_gray is not None, "Изображения не загружены."

# TODO: инициализируйте ORB-детектор
orb = cv2.ORB_create(nfeatures=5000)

# TODO: найдите ключевые точки и дескрипторы
kp1, des1 = orb.detectAndCompute(imgL_gray, None)
kp2, des2 = orb.detectAndCompute(imgR_gray, None)

# TODO: сопоставьте дескрипторы (matcher — Brute Force по Hamming)
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = matcher.match(des1, des2)

# Отсортируем по расстоянию
matches = sorted(matches, key=lambda x: x.distance)


## Визуализация совпадений между изображениями

Посмотрим, насколько хорошо сработало сопоставление признаков.


In [ ]:
img_matches = cv2.drawMatches(
    imgL_gray, kp1,
    imgR_gray, kp2,
    matches[:50], None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)

show_img(img_matches, title="Лучшие 50 совпадений")


## Поиск фундаментальной матрицы и фильтрация инлайеров

Фундаментальная матрица \( F \) описывает эпиполярную геометрию между двумя изображениями.
Рассчитаем её по совпадающим точкам с помощью RANSAC.

Вы:
- Извлечёте координаты точек по найденным матчам;
- Вычислите \( F \);
- Отфильтруете инлайеры.


In [ ]:
pts1 = np.float32([kp1[m.queryIdx].pt for m in matches])
pts2 = np.float32([kp2[m.trainIdx].pt for m in matches])

# TODO: найдите фундаментальную матрицу и маску инлайеров
F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC)

print("F =", F)


## Визуализация эпиполярных линий

Построим эпиполярные линии на одном изображении, соответствующие точкам на другом.

Для этого используем:
$$
l' = F \cdot x, \quad \text{или} \quad l = F^T \cdot x'
$$

И построим линии на изображении по уравнению \( ax + by + c = 0 \).


In [ ]:
def draw_epipolar_lines(img1, img2, pts1, pts2, F):
    """
    Рисует эпиполярные линии на img2, соответствующие точкам на img1.
    """
    lines = cv2.computeCorrespondEpilines(pts1.reshape(-1, 1, 2), 1, F)
    lines = lines.reshape(-1, 3)

    img1_color = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2_color = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)

    for (r, pt1, pt2) in zip(lines, pts1, pts2):
        color = tuple(np.random.randint(0, 255, 3).tolist())
        x0, y0 = map(int, pt2)
        a, b, c = r
        x1, y1 = 0, int(-c / b) if b != 0 else 0
        x2, y2 = img2.shape[1], int(-(a * x2 + c) / b) if b != 0 else 0

        cv2.line(img2_color, (x1, y1), (x2, y2), color, 1)
        cv2.circle(img1_color, (x0, y0), 4, color, -1)

    return img1_color, img2_color


# TODO: оставьте только инлайеры
pts1_inl = pts1[mask.ravel() == 1]
pts2_inl = pts2[mask.ravel() == 1]

img1_epi, img2_epi = draw_epipolar_lines(imgL_gray, imgR_gray,
                                         pts1_inl, pts2_inl, F)

show_row([img1_epi, img2_epi], ["Точки (L)", "Эпиполярные линии (R)"], figsize=(10, 5))


## Ректификация изображений (выравнивание по горизонтали)

Ректификация преобразует изображения так, чтобы эпиполярные линии стали строго горизонтальными.

Вы:
- Получите гомографии для ректификации;
- Преобразуете изображения;
- Проверите, что линии выровнены.


In [ ]:
def rectify_uncalibrated(img1, img2, pts1, pts2, F):
    """
    Ректифицирует изображения с использованием фундаментальной матрицы.
    """
    h, w = img1.shape[:2]

    # TODO: найдите гомографии H1 и H2
    retval, H1, H2 = cv2.stereoRectifyUncalibrated(
        pts1, pts2, F, imgSize=(w, h)
    )

    assert retval, "Ректификация не удалась"

    # TODO: преобразуйте изображения
    img1_rect = cv2.warpPerspective(img1, H1, (w, h))
    img2_rect = cv2.warpPerspective(img2, H2, (w, h))

    return img1_rect, img2_rect, H1, H2


imgL_rect, imgR_rect, H1, H2 = rectify_uncalibrated(
    imgL_gray, imgR_gray, pts1_inl, pts2_inl, F
)

show_row([imgL_rect, imgR_rect], ["Левое (ректифицировано)", "Правое (ректифицировано)"], figsize=(10, 4))


## Проверка: эпиполярные линии после ректификации

Проверьте, что после ректификации соответствующие точки лежат на одной горизонтали.
Для этого нарисуем горизонтальные сетки поверх изображений.


In [ ]:
def show_with_grid(img, n_lines=15, color=(0, 255, 0)):
    img_color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    h, w = img.shape
    for y in np.linspace(0, h, n_lines, dtype=int):
        cv2.line(img_color, (0, y), (w, y), color, 1)
    return img_color

show_row(
    [show_with_grid(imgL_rect), show_with_grid(imgR_rect)],
    ["Сетка на левом", "Сетка на правом"], figsize=(10, 4)
)


## Получение карты диспаритета

Используем OpenCV `StereoSGBM` для расчёта карты смещений (disparity) между парой изображений.

Чем ближе объект — тем больше disparity.


In [ ]:
# TODO: настройте параметры стерео-алгоритма
stereo = cv2.StereoSGBM_create(
    minDisparity=0,
    numDisparities=64,  # должно делиться на 16
    blockSize=7,
    P1=8*3*7**2,
    P2=32*3*7**2,
    mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
)

disp_raw = stereo.compute(imgL_rect, imgR_rect).astype(np.float32) / 16.0

# Преобразуем недопустимые значения в NaN
disp = np.where(disp_raw <= 0, np.nan, disp_raw)

plt.figure(figsize=(8, 4))
plt.imshow(disp, cmap="plasma")
plt.colorbar(label="disparity")
plt.title("Карта диспаритета")
plt.axis("off")
plt.show()


## Увеличенный фрагмент карты диспаритета

Рассмотрим фрагмент изображения, чтобы лучше увидеть разницу в глубине между объектами.


In [ ]:
def show_crop_with_disp(img, disp, x0, y0, dx, dy):
    img_crop = img[y0:y0+dy, x0:x0+dx]
    disp_crop = disp[y0:y0+dy, x0:x0+dx]

    show_row([img_crop, disp_crop],
             ["Фрагмент изображения", "disparity-фрагмент"],
             figsize=(10, 4), cmap2="plasma")


show_crop_with_disp(imgL_rect, disp, x0=150, y0=80, dx=150, dy=100)


## Визуализация облака точек из disparity

На основе значений disparity можно восстановить относительную глубину сцены.

В этом задании вы:
- Преобразуете disparity в координаты \( X, Y, Z \);
- Отобразите результат в 3D.


In [ ]:
from mpl_toolkits.mplot3d import Axes3D  # noqa
from matplotlib import cm

def disparity_to_point_cloud(disp, step=2, max_points=8000):
    """
    Преобразует карту disparity в облако точек (X, Y, Z), где
    Z ~ 1 / disparity (относительная глубина).

    step — прореживание (чем больше, тем реже точки);
    max_points — максимальное число отображаемых точек.
    """
    h, w = disp.shape

    # Сетка координат
    ys, xs = np.mgrid[0:h:step, 0:w:step]
    ds = disp[0:h:step, 0:w:step]

    mask = ~np.isnan(ds)
    xs = xs[mask].astype(np.float32)
    ys = ys[mask].astype(np.float32)
    ds = ds[mask].astype(np.float32)

    if len(xs) == 0:
        raise RuntimeError("Нет валидных значений disparity.")

    # TODO: глубина обратно пропорциональна disparity
    Z = 1.0 / (ds + 1e-6)
    Z = (Z - Z.min()) / (Z.max() - Z.min() + 1e-6)

    if len(xs) > max_points:
        idx = np.random.choice(len(xs), size=max_points, replace=False)
        xs, ys, Z = xs[idx], ys[idx], Z[idx]

    return xs, ys, Z


# Построение облака точек
X, Y, Z = disparity_to_point_cloud(disp)

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
p = ax.scatter(X, Y, Z, c=Z, cmap="plasma", s=1)

ax.set_xlabel("X (pixels)")
ax.set_ylabel("Y (pixels)")
ax.set_zlabel("Relative depth")
ax.set_title("3D-точки из disparity")

fig.colorbar(p, label="Relative depth")
plt.show()
